# Cafram2 Tutorial

This tutorial should help you to get into cafram, with a tutorial application. For this purpose, let's imagine a simple app that manage git repos.

# Base Application Example

Let's start with a simple example. The idea is to store the whole program configuration in a simple configuration. For demonstration purpose, we We represent our application configuration into `app_config`, a serializable dict. Then we create an inherited `NodeConfDict` `App` class, that will represent our application.

In [1]:

from cafram2.mixins.tree import ConfDictMixin, NodeConfDict
from cafram2.nodes import Node2 as Node

app_config = {
        "namespace": "username",        
        "config": {
            "enabled": True,
            "desc": "This is a description",
            "remote": "http://gitlab.com/",
            "branches": [
                    "main"
                ],
            "var_files": [
                "first.env",
                "second.env",
            ],
        },
        "repos": [
            {
                "name": "my_repo1.git",
            },
            {
                "name": "my_repo2.git",
                "branches": [
                    "main",
                    "develop"
                ],
                "enabled": False,
            },
            "my_repo3.git",
        ],
           
    }

# You can either configure manually your class


class MyApp(Node):

    _node_conf = [
        {
            "mixin": ConfDictMixin,
        }
    ]

# Or use the preset Node class NodeConfDict
class MyApp(NodeConfDict):
    pass

app = MyApp(payload=app_config)


We now have our Node instanciated with the following configuration. Let's use some internal cafram helpers to represent it in yaml:

In [2]:
from cafram2.utils import to_json, from_json, to_yaml, from_yaml

print(to_yaml(app_config))

namespace: username
config:
  enabled: true
  desc: This is a description
  remote: http://gitlab.com/
  branches:
  - main
  var_files:
  - first.env
  - second.env
repos:
- name: my_repo1.git
- name: my_repo2.git
  branches:
  - main
  - develop
  enabled: false
- my_repo3.git



At this stage, we have an App initialized with a config. Let's discover how to use it.

# Mixin: ConfDictMixin

Now we can access to dict sub elements:

In [3]:
#pprint(payload_app)

print ("App status:")
print (f"  Namespace: {app.namespace}")
print (f"  Default remote: {app.config.remote}")
print (f"  Is enabled: {app.config.enabled}")

print (f"Repos to manage: {app.repos}")


App status:
  Namespace: username
  Default remote: http://gitlab.com/
  Is enabled: True
Repos to manage: <cafram2.mixins.tree.NodeConfList object at 0x7facd2eafe20>


Everything is good so far, but we have an issue with `app.repos` node, it shows object instead of content.

It is important to notice containers (list and dicts) does not works the same way as immutable values, they return a subclass of `NodeConf` (which is a descendant of `CaframNode->Node->NodePayload`). The way to retrieve the content of the node is via the `get_value()` method of `ConfDictMixin`. This behavior can be changed later thanks to the `children` parameter of `ConfDictMixin`.

In [4]:
print(f"Repos to manage: {app.repos.conf.get_value()}")

Repos to manage: [{'name': 'my_repo1.git'}, {'name': 'my_repo2.git', 'branches': ['main', 'develop'], 'enabled': False}, 'my_repo3.git']


ConfDictMixin also provide an alias to retrieve the value, via the `value` attribute:

In [5]:
app.repos.value == app.repos.conf.get_value()

True

However, the `value` attribute behavior may not be desired, it can be disabled by setting `value_alias: None` in the Mixin config.

In [6]:
print("Repos to manage:")

for repo_conf in app.repos.value:
    print (f"  - {repo_conf}")

Repos to manage:
  - {'name': 'my_repo1.git'}
  - {'name': 'my_repo2.git', 'branches': ['main', 'develop'], 'enabled': False}
  - my_repo3.git


Let's make it a bit nicer:

In [7]:
print("Repos to manage:")

for repo_conf in app.repos.value:
    
    # As we have different types of values in our list
    status = True
    if isinstance(repo_conf, dict):
        name = repo_conf["name"]
        status = repo_conf.get("enabled", True)
    else:
        name = repo_conf
    status = "enabled" if status else "disabled"
    
    print (f"  - {name} ({status})")

Repos to manage:
  - my_repo1.git (enabled)
  - my_repo2.git (disabled)
  - my_repo3.git (enabled)


Cool, we have a list of repos to manage. At this stage, let's see how we can extract some parts of the app state:

In [8]:
print(app.conf.to_json())

{
  "namespace": "username",
  "config": {
    "enabled": true,
    "desc": "This is a description",
    "remote": "http://gitlab.com/",
    "branches": [
      "main"
    ],
    "var_files": [
      "first.env",
      "second.env"
    ]
  },
  "repos": [
    {
      "name": "my_repo1.git"
    },
    {
      "name": "my_repo2.git",
      "branches": [
        "main",
        "develop"
      ],
      "enabled": false
    },
    "my_repo3.git"
  ]
}


In [9]:
print(app.config.var_files.conf.to_yaml())

- first.env
- second.env



## Nodes relationships

Now we've seen how to extract data, let's explore node relationship. Let's start by children and the parents:

In [10]:
from pprint import pprint

print("=> Get direct children, containers are Nodes:")
pprint (app.conf.get_children())

print("\n=> Get children reccursively, containers are values:")
pprint (app.conf.get_children(-1))

print("\n=> This also works on sub nodes:")
pprint (app.config.conf.get_children())

print("\n=> And recursively on sub nodes:")
pprint (app.config.conf.get_children(-1))

=> Get direct children, containers are Nodes:
{'config': <cafram2.mixins.tree.NodeConfDict object at 0x7facd2eaff10>,
 'namespace': 'username',
 'repos': <cafram2.mixins.tree.NodeConfList object at 0x7facd2eafe20>}

=> Get children reccursively, containers are values:
{'config': {'branches': ['main'],
            'desc': 'This is a description',
            'enabled': True,
            'remote': 'http://gitlab.com/',
            'var_files': ['first.env', 'second.env']},
 'namespace': 'username',
 'repos': [{'name': 'my_repo1.git'},
           {'branches': ['main', 'develop'],
            'enabled': False,
            'name': 'my_repo2.git'},
           'my_repo3.git']}

=> This also works on sub nodes:
{'branches': <cafram2.mixins.tree.NodeConfList object at 0x7facd2eafb80>,
 'desc': 'This is a description',
 'enabled': True,
 'remote': 'http://gitlab.com/',
 'var_files': <cafram2.mixins.tree.NodeConfList object at 0x7facd2eafb50>}

=> And recursively on sub nodes:
{'branches': ['main

This way can be discovered the app structure, and you can navigate between nodes. Let's select a node to try parents relationship now:

In [24]:
sub_conf = app.config.var_files
sub_conf


Let's find out parents:

In [26]:
parents = sub_conf.conf.get_parents()
pprint(parents)

Where nodes are actually:

In [27]:
print ("=> First parent:")
print(parents[0].conf.to_yaml())

print ("\n=> Second parent:")
print(parents[1].conf.to_yaml())

=> First parent:
enabled: true
desc: This is a description
remote: http://gitlab.com/
branches:
- main
var_files:
- first.env
- second.env


=> Second parent:
namespace: username
config:
  enabled: true
  desc: This is a description
  remote: http://gitlab.com/
  branches:
  - main
  var_files:
  - first.env
  - second.env
repos:
- name: my_repo1.git
- name: my_repo2.git
  branches:
  - main
  - develop
  enabled: false
- my_repo3.git



We've seen most basic usage of the NodeDictConf, but there is more. To find out what is achievable, checkout internal mixin documentation:

In [29]:
sub_conf.conf.doc()

Documentation for: cafram2.mixins.tree.ConfListMixin
  Usage:
    Conf mixin that manage a serializable list of values
    
        Usage:
          ConfListMixin(node_ctrl, mixin_conf=None)
          ConfListMixin(node_ctrl, mixin_conf=[ConfDictMixin])

    
  Mixins inheritance:
    - builtins.object
    - cafram2.common.CaframObj
    - cafram2.common.CaframMixin
    - cafram2.mixins.BaseMixin
    - cafram2.mixins.tree.PayloadMixin
    - cafram2.mixins.tree.HierParentMixin
    - cafram2.mixins.tree.ConfMixin
    - cafram2.mixins.tree.HierChildrenMixin
    - cafram2.mixins.tree.ConfContainerMixin
    - cafram2.mixins.tree.ConfListMixin

  Parameters:
    children: True
    children_param: children
    mixin_conf: {'mixin': <class 'cafram2.mixins.tree.ConfListMixin'>}
    name: conf
    name_param: payload
    parent_param: parent
    value: ['first.env', 'second.env']
    value_alias: value

  Methods:
    add_child(child, index=None, alias=True):
      Add a new child to mixin
    do

## Data validation

TODO

# Understand mixins API

There are some pythonic ways to see how mixins works. It is possible to get a technical overview of config parameters, but also mixins methods and other informations. To get more information on `ConfDictMixin` :

In [19]:
from cafram2.tools import MixinLoader

MixinLoader(ConfDictMixin).doc()

Default mixin name: conf (instead of debug)

Documentation for: cafram2.tools.MixinLoaderInst
  Usage:
    <Missing>
  Mixins inheritance:
    - builtins.object
    - cafram2.common.CaframObj
    - cafram2.common.CaframMixin
    - cafram2.mixins.BaseMixin
    - cafram2.mixins.tree.PayloadMixin
    - cafram2.mixins.tree.HierParentMixin
    - cafram2.mixins.tree.ConfMixin
    - cafram2.mixins.tree.HierChildrenMixin
    - cafram2.mixins.tree.ConfContainerMixin
    - cafram2.mixins.tree.ConfDictMixin
    - cafram2.tools.MixinLoaderInst

  Parameters:
    children: True
    children_param: children
    mixin_conf: {'mixin': <class 'cafram2.tools.MixinLoader.__init__.<locals>.MixinLoaderInst'>}
    name: debug
    name_param: payload
    parent_param: parent
    value: None
    value_alias: value

  Methods:
    add_child(child, index=None, alias=True):
      Add a new child to mixin
    doc(details=False):
      Show mixin internal documentation
    dump(stdout=True, details=False, ignore=N

Every mixin should provide such informations. However, if you want to get an overview of a mixin instance, while you're debugging, you can use mixin.dump(). It also works on NodeCtrl.